# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [1]:
import pandas as pd
import numpy as np

#### Open the ` Employee.csv` file and name your dataset `employee`.

In [3]:
employee = pd.read_csv('data/Employee.csv')
employee

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55
5,Eva,Sales,Bachelor,F,associate,2,55
6,Carlos,IT,Master,M,VP,8,70
7,Pedro,IT,Phd,M,associate,7,60
8,Ana,HR,Master,F,VP,8,70


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with groupby and the other way is with pivot tables. Do it both ways. 


In [4]:
# Method 1: groupby
employee.groupby('Department').agg({'Salary':'mean'})

,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


In [5]:
# Method 2: pivot_table
employee.pivot_table(index='Department', values = ['Salary'], aggfunc = ['mean'])

,mean
,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [11]:
employee_dt_salary = employee.pivot_table(index=['Department','Title'], values = ['Salary'], aggfunc = ['mean']).reset_index()
employee_dt_salary.columns = ['Department', 'Title', 'Average Salary']
employee_dt_salary

,Department,Title,Average Salary
0,HR,VP,70.0
1,HR,analyst,32.5
2,IT,VP,70.0
3,IT,analyst,32.5
4,IT,associate,60.0
5,Sales,associate,55.0


#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [40]:
employee_ys_title = employee.pivot_table(index=['Title'], columns=['Years'], values = ['Salary'], fill_value = 0).reset_index()
employee_ys_title

Title Salary                    
Years                 1   2   3   4   7   8
0             VP      0   0   0   0   0  70
1        analyst     35  30   0  35   0   0
2      associate      0  55  55   0  60   0

#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [38]:
# use pd.series.nunique to count distinct vals in Pandas aggregation

employee_dt_mean = employee.pivot_table(index=['Department','Title'], values = ['Salary', 'Name'], aggfunc = {'Salary': np.mean, 'Name': pd.Series.nunique}).reset_index()
employee_dt_mean

,Department,Title,Name,Salary
0,HR,VP,1,70.0
1,HR,analyst,2,32.5
2,IT,VP,1,70.0
3,IT,analyst,2,32.5
4,IT,associate,1,60.0
5,Sales,associate,2,55.0


#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [31]:
employee_stats = employee.pivot_table(index=['Department','Title'], values = 'Salary', aggfunc= ['mean','median',min, max, 'std']).reset_index()
employee_stats.columns = ['Department','Title','Mean Salary','Median Salary', 'Minimum Salary', 'Maximum Salary', 'St Dev Salary']
employee_stats

,Department,Title,Mean Salary,Median Salary,Minimum Salary,Maximum Salary,St Dev Salary
0,HR,VP,70.0,70.0,70,70,NaN
1,HR,analyst,32.5,32.5,30,35,3.535534
2,IT,VP,70.0,70.0,70,70,NaN
3,IT,analyst,32.5,32.5,30,35,3.535534
4,IT,associate,60.0,60.0,60,60,NaN
5,Sales,associate,55.0,55.0,55,55,0.000000


<b>Comments:</b> we can see that the salaries of VP and Analyst do not change per department, while the salary of associate between IT and Sales has a difference of 5K. As there is only one value for three rows, the St. Deviation  appears as null. 

#### Based on your comments, fill the missing values with an appropriate value.

In [32]:
employee_stats['St Dev Salary'] = employee_stats['St Dev Salary'].fillna(0)
employee_stats

,Department,Title,Mean Salary,Median Salary,Minimum Salary,Maximum Salary,St Dev Salary
0,HR,VP,70.0,70.0,70,70,0.000000
1,HR,analyst,32.5,32.5,30,35,3.535534
2,IT,VP,70.0,70.0,70,70,0.000000
3,IT,analyst,32.5,32.5,30,35,3.535534
4,IT,associate,60.0,60.0,60,60,0.000000
5,Sales,associate,55.0,55.0,55,55,0.000000


#### The stake holders want to know for each department the number of employees and how much money is spend on salaries. Could you provide that information? 

In [35]:
employee_dn_salary = employee.pivot_table(index=['Department'], values=['Name','Salary'], aggfunc={'Name':pd.Series.nunique, 'Salary':sum}).reset_index()
employee_dn_salary

,Department,Name,Salary
0,HR,3,135
1,IT,4,195
2,Sales,2,110


#### For each Department and Title, we want to know the the total years of work experience and the mean salary.

In [36]:
employee_dt_expsal = employee.pivot_table(index=['Department', 'Title'], values=['Years', 'Salary'], aggfunc={'Years':sum, 'Salary': 'mean'}).reset_index()
employee_dt_expsal

,Department,Title,Salary,Years
0,HR,VP,70.0,8
1,HR,analyst,32.5,6
2,IT,VP,70.0,8
3,IT,analyst,32.5,3
4,IT,associate,60.0,7
5,Sales,associate,55.0,5


### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.

In [54]:
# filtering dfs by max rows work like df.loc[df['sq_resid']!=df['sq_resid'].max()]
# what we want here is to work with a single column, so adapt the df filter to it
# compare against the max val and keep the rest, and round it to 2 decimals

def mean_salary_sans_maxima(col):
    mean_col = round(col[col!=col.max()].mean(), 2)
    return mean_col

employee.pivot_table(index = ['Department'], aggfunc = {'Salary':mean_salary_sans_maxima}).reset_index()

,Department,Salary
0,HR,32.50
1,IT,41.67
